In [12]:
import warnings

warnings.filterwarnings("ignore")

# Grid

The `grid` module defines the top level of `pytileproj`'s tiling system concept. A grid is a collection of tiling systems, which share a tiling concepts, but differ in their projection (however, the projection should be from the same family). An example is the [Equi7Grid](https://github.com/TUW-GEO/Equi7Grid) projection, which is a collection of seven continental equidistant azimuthal projections. 

## Regular grid

Within the `grid` module, the class `RegularGrid` contains a collection of `RegularProjTilingSystem` as on-the-fly generated class variables. Next to a `RegularProjTilingSystem`, the `RegularGrid` class was designed as the main entry point to handle projected tiling systems and to create them in a flexible and user-friendly way. 

### Initialisation

To create a `RegularGrid` object, one way is to define all the objects directly in Python. In principle, we can start with creating a simple `RegularGrid` object by using the same regular projected tiling system as defined in the [tiling system documentation](https://tuw-geo.github.io/pytileproj/guides/tiling_systemt.html):

In [ ]:
from pathlib import Path

from pytileproj import (
    ProjSystemDefinition,
    RegularProjTilingSystem,
    RegularTilingDefinition,
)

name = "e7eu"
epsg = 27704
min_xy = (0, 0)
max_xy = None  # None will be filled with the projection bounds
axis_orientation = ("E", "S")

# this argument is only optional, but in case the EPSG API is down,
# it needs to be provided
proj_zone_geog = Path("../../tests/data/eu_zone.parquet")

proj_def = ProjSystemDefinition(
    name=name,
    crs=epsg,
    min_xy=min_xy,
    max_xy=max_xy,
    axis_orientation=axis_orientation,
    proj_zone_geog=proj_zone_geog,
)
tiling_defs = {
    1: RegularTilingDefinition(name="my_coarse_tiling", tile_shape=300_000),
    2: RegularTilingDefinition(name="my_fine_tiling", tile_shape=100_000),
}
rpts = RegularProjTilingSystem.from_sampling(
    {1: 1_000, 2: 10}, proj_def=proj_def, tiling_defs=tiling_defs
)

With this, we are now able to initialise a `RegularGrid` object:

In [14]:
from pytileproj import RegularGrid

rgrid = RegularGrid(**{name: rpts})

However, if we work with a grid system with several tiling systems beneath, then defining everything within Python may become tedious. Therefore, there are several classmethods, which allow to generate a `RegularGrid` object. 

`from_sampling` is exactly the same method as for a `RegularProjTilingSystem`, but now working with multiple `RegularTilingDefinition` instances. Here is an example:

In [15]:
RegularGrid.from_sampling(
    {1: 1_000, 2: 10}, proj_defs={name: proj_def}, tiling_defs=tiling_defs
)

RegularGrid(system_order=['e7eu'], e7eu=RegularProjTilingSystem(crs=27704, proj_zone_geog=GeogGeom(geom=<POLYGON ((50.065 62.864, 50.061 62.964, 50.058 63.064, 50.055 63.164, 50.05...>, crs=<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
), name='e7eu', tilings={1: RegularTiling(name='my_coarse_tiling', extent=(0.0, 0.0, 8400000.0, 5700000.0), sampling=1000.0, tile_shape=(300000.0, 300000.0), tiling_level=1, axis_orientation=('E', 'S')), 2: RegularTiling(name='my_fine_tiling', extent=(0.0, 0.0, 8300000.0, 5600000.0), sampling=10.0, tile_shape=(100000.0, 100000.0), tiling_level=2, axis_orientation=('E', 'S'))}))

#### Export and import grid definitions

Another possibility is to start from a more generic regular grid definition. As an example, the core tiling system definitions of a `RegularGrid` object can be exported to a JSON file via

In [16]:
from pathlib import Path

grid_def_path = Path("my_grid_def.json")
rgrid.to_grid_def(grid_def_path)

If we open it, it looks like this:

In [17]:
import json
import pprint

with grid_def_path.open() as f:
    grid_def = json.load(f)

pprint.pprint(grid_def)  # noqa: T203

{'proj_defs': {'e7eu': {'axis_orientation': ['E', 'S'],
                        'crs': 27704,
                        'max_xy': [8400000.0, 5700000.0],
                        'min_xy': [0.0, 0.0],
                        'name': 'e7eu',
                        'proj_zone_geog': {'crs': 'GEOGCRS["WGS '
                                                  '84",ENSEMBLE["World '
                                                  'Geodetic System 1984 '
                                                  'ensemble",MEMBER["World '
                                                  'Geodetic System 1984 '
                                                  '(Transit)"],MEMBER["World '
                                                  'Geodetic System 1984 '
                                                  '(G730)"],MEMBER["World '
                                                  'Geodetic System 1984 '
                                                  '(G873)"],MEMBER["World '
                   

Now we can use this file to recreate the object by specifying the desired sampling. Like this it is convenient to share a grid definition with other people.

In [18]:
RegularGrid.from_grid_def(grid_def_path, {1: 1_000, 2: 10})

RegularGrid(system_order=['e7eu'], e7eu=RegularProjTilingSystem(crs=27704, proj_zone_geog=GeogGeom(geom=<POLYGON ((50.065 62.864, 50.061 62.964, 50.058 63.064, 50.055 63.164, 50.05...>, crs=<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
), name='e7eu', tilings={1: RegularTiling(name='my_coarse_tiling', extent=(0.0, 0.0, 8400000.0, 5700000.0), sampling=1000.0, tile_shape=(300000.0, 300000.0), tiling_level=1, axis_orientation=('E', 'S')), 2: RegularTiling(name='my_fine_tiling', extent=(0.0, 0.0, 8400000.0, 5700000.0), sampling=10.0, tile_shape=(100000.0, 100000.0), tiling_level=2, axis_orientation=('E', 'S'))}))

If we want to be more explicit, there is also the option to export all class properties to a JSON file. This includes the sampling, thus allowing to exactly replicate the object, from which the JSON file was created.

In [19]:
grid_path = Path("my_grid.json")
rgrid.to_file(grid_path)

Now the JSON file looks like this:

In [20]:
with grid_path.open() as f:
    grid = json.load(f)

pprint.pprint(grid)  # noqa: T203

{'e7eu': {'crs': 27704,
          'name': 'e7eu',
          'proj_zone_geog': {'crs': 'GEOGCRS["WGS 84",ENSEMBLE["World Geodetic '
                                    'System 1984 ensemble",MEMBER["World '
                                    'Geodetic System 1984 '
                                    '(Transit)"],MEMBER["World Geodetic System '
                                    '1984 (G730)"],MEMBER["World Geodetic '
                                    'System 1984 (G873)"],MEMBER["World '
                                    'Geodetic System 1984 '
                                    '(G1150)"],MEMBER["World Geodetic System '
                                    '1984 (G1674)"],MEMBER["World Geodetic '
                                    'System 1984 (G1762)"],MEMBER["World '
                                    'Geodetic System 1984 '
                                    '(G2139)"],MEMBER["World Geodetic System '
                                    '1984 (G2296)"],ELLIPSOID["WGS '
    

'POLYGON ((50.064526460000195 '
                                     '62.86427365900016, 50.06124290000008 '
                                     '62.96416449000003, 50.05793463300017 '
                                     '63.06405525000014, 50.05460138200016 '
                                     '63.1639459380001, 50.05124286800009 '
                                     '63.26383655400008, 50.04785880800006 '
                                     '63.36372709600005, 50.04444891300011 '
                                     '63.46361756500005, 50.04101289100004 '
                                     '63.56350795900016, 50.03755044600018 '
                                     '63.66339827700011, 50.034061277000035 '
                                     '63.76328852000012, 50.030545077000056 '
                                     '63.86317868500009, 50.02700153600006 '
                                     '63.96306877300003, 50.02343033799997 '
                                     '64.06

And the object can be recreated in the same manner:

In [21]:
RegularGrid.from_file(grid_path)

RegularGrid(system_order=['e7eu'], e7eu=RegularProjTilingSystem(crs=27704, proj_zone_geog=GeogGeom(geom=<POLYGON ((50.065 62.864, 50.061 62.964, 50.058 63.064, 50.055 63.164, 50.05...>, crs=<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
), name='e7eu', tilings={1: RegularTiling(name='my_coarse_tiling', extent=(0.0, 0.0, 8400000.0, 5700000.0), sampling=1000.0, tile_shape=(300000.0, 300000.0), tiling_level=1, axis_orientation=('E', 'S')), 2: RegularTiling(name='my_fine_tiling', extent=(0.0, 0.0, 8300000.0, 5600000.0), sampling=10.0, tile_shape=(100000.0, 100000.0), tiling_level=2, axis_orientation=('E', 'S'))}))

#### Tiling system order

An optional argument available for all initalisation methods is `system_order`, which defines which tiling system should be used and how their order should be. This is important if users only want to work with a subset of the available tiling systems or if projection zones overlap and there should be a preference when querying for tiles and coordinates. In this example we add two tiling systems to a regular grid and define their order:

In [ ]:
af_def = ProjSystemDefinition(
    name="e7af",
    crs=27701,
    min_xy=min_xy,
    max_xy=max_xy,
    axis_orientation=axis_orientation,
    proj_zone_geog=Path("../../tests/data/af_zone.parquet"),
)
eu_def = ProjSystemDefinition(
    name="e7eu",
    crs=27704,
    min_xy=min_xy,
    max_xy=max_xy,
    axis_orientation=axis_orientation,
    proj_zone_geog=proj_zone_geog,
)
af_rpts = RegularProjTilingSystem.from_sampling(
    {1: 1_000, 2: 10}, proj_def=af_def, tiling_defs=tiling_defs
)
eu_rpts = RegularProjTilingSystem.from_sampling(
    {1: 1_000, 2: 10}, proj_def=eu_def, tiling_defs=tiling_defs
)

rgrid = RegularGrid(
    system_order=["e7eu", "e7af"], **{rpts.name: rpts for rpts in [af_rpts, eu_rpts]}
)

### Supported samplings

If you are not sure, what samplings you can use for your grid design, you can fetch them directly via the `allowed_samplings` method with the desired tile size as an argument:

In [23]:
RegularGrid.allowed_samplings(300)

[1, 2, 3, 4, 5, 6, 10, 12, 15, 20, 25, 30, 50, 60, 75, 100, 150, 300]

### Tiling system access

The regular projected tiling system can then be accessed with:

In [24]:
rgrid.e7eu  # ty: ignore[unresolved-attribute]

RegularProjTilingSystem(crs=27704, proj_zone_geog=GeogGeom(geom=<POLYGON ((50.065 62.864, 50.061 62.964, 50.058 63.064, 50.055 63.164, 50.05...>, crs=<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
), name='e7eu', tilings={1: RegularTiling(name='my_coarse_tiling', extent=(0.0, 0.0, 8400000.0, 5700000.0), sampling=1000.0, tile_shape=(300000.0, 300000.0), tiling_level=1, axis_orientation=('E', 'S')), 2: RegularTiling(name='my_fine_tiling', extent=(0.0, 0.0, 8300000.0, 5600000.0), sampling=10.0, tile_shape=(100000.0, 100000.0), tiling_level=2, axis_orientation=('E', 'S'))})

or equivalently,

In [25]:
rgrid[name]

RegularProjTilingSystem(crs=27704, proj_zone_geog=GeogGeom(geom=<POLYGON ((50.065 62.864, 50.061 62.964, 50.058 63.064, 50.055 63.164, 50.05...>, crs=<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich
), name='e7eu', tilings={1: RegularTiling(name='my_coarse_tiling', extent=(0.0, 0.0, 8400000.0, 5700000.0), sampling=1000.0, tile_shape=(300000.0, 300000.0), tiling_level=1, axis_orientation=('E', 'S')), 2: RegularTiling(name='my_fine_tiling', extent=(0.0, 0.0, 8300000.0, 5600000.0), sampling=10.0, tile_shape=(100000.0, 100000.0), tiling_level=2, axis_orientation=('E', 'S'))})

In case there are several tiling systems defining the regular grid and you want to know which tiling system(s) to use for a certain location, then you can use `get_systems_from_coord` or `get_systems_from_lonlat`. Here is an example with a coordinate inside the regular grid

In [26]:
rgrid.get_systems_from_lonlat(16, 48)

[RegularProjTilingSystem(crs=27704, proj_zone_geog=GeogGeom(geom=<POLYGON ((50.065 62.864, 50.061 62.964, 50.058 63.064, 50.055 63.164, 50.05...>, crs=<Geographic 2D CRS: EPSG:4326>
 Name: WGS 84
 Axis Info [ellipsoidal]:
 - Lat[north]: Geodetic latitude (degree)
 - Lon[east]: Geodetic longitude (degree)
 Area of Use:
 - name: World.
 - bounds: (-180.0, -90.0, 180.0, 90.0)
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich
 ), name='e7eu', tilings={1: RegularTiling(name='my_coarse_tiling', extent=(0.0, 0.0, 8400000.0, 5700000.0), sampling=1000.0, tile_shape=(300000.0, 300000.0), tiling_level=1, axis_orientation=('E', 'S')), 2: RegularTiling(name='my_fine_tiling', extent=(0.0, 0.0, 8300000.0, 5600000.0), sampling=10.0, tile_shape=(100000.0, 100000.0), tiling_level=2, axis_orientation=('E', 'S'))})]

and outside

In [27]:
rgrid.get_systems_from_lonlat(-100, 48)

GeomOutOfZoneError: The given Point ('POINT (-100 48)') is not within the zone boundaries of the projection.

### Geographic coordinate transformation

A regular grid object also allows to transform geographic coordinates to grid coordinates. Usually, there is a unique result. But in case projection zones of different tiling systems overlap, coordinates are returned for each intersecting tiling system - with the order defined by `system_order`.

In [28]:
rgrid.lonlat_to_xy(16, 48)

{'e7eu': ProjCoord(x=5240688.087435639, y=1597809.919837502, crs=<Projected CRS: EPSG:27704>
 Name: WGS 84 / Equi7 Europe
 Axis Info [cartesian]:
 - E[east]: Easting (metre)
 - N[north]: Northing (metre)
 Area of Use:
 - name: Europe including Russia west of the Ural Mountains.
 - bounds: (-42.52, 29.24, 51.73, 83.67)
 Coordinate Operation:
 - name: Equi7 projection - Europe
 - method: Azimuthal Equidistant
 Datum: World Geodetic System 1984 ensemble
 - Ellipsoid: WGS 84
 - Prime Meridian: Greenwich
 )}

In [29]:
grid_path.unlink(missing_ok=True)
grid_def_path.unlink(missing_ok=True)